In [160]:
import os
import pandas as pd
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 
import gensim
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
from bs4 import BeautifulSoup
import urllib.request
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize 
import csv
#Getting data from the link
#response=urllib.request.urlopen('http://php.net/')
#html=response.read()
#Soup=BeautifulSoup(html,"html5lib")
#inptxt=Soup.get_text(strip = True)
#Removing stopwords from raw data
#noise_list = ["The","a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
noise_list=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
lookup_dict = {'rt':'Retweet', 'dm':'direct message', "awsm" : "awesome", "wrt" : "with respect to", "pm" :"personal message", "UI" :"User Interface", "IDE" :"Integrated Development Environment"}
data = pd.read_csv("Connect_TC.csv")
inptxt = data.set_index('DS_TEST_ID')
pd.options.display.max_colwidth = 0
inptxt = inptxt.replace('\n',' ',regex=True)
inptxt1=""
lemt2=""
lemt3=""
lem=WordNetLemmatizer()
for row in inptxt.iterrows():
    row=re.sub('\.',' ', str(row))
def removenoise(input_text):
    words = input_text.split() 
    noise_free_words = [word for word in words if word not in noise_list] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text
def removeregex(input_text, regex_pattern):
    urls = re.finditer(regex_pattern, input_text) 
    for i in urls: 
        input_text = re.sub(i.group().strip(), '', input_text)
    return input_text
#regex_pattern='[^\s\w]'
regex_pattern = "[\w]*"
#[!@#\$%\^&\*\)\(\+-=\?]
def remreg(input_text):
    xy=re.sub('[^a-zA-Z0-9]', '', input_text)
    return xy
def lookupwords(input_text):
    words = input_text.split() 
    new_words = [] 
    for word in words:
        if word.lower() in lookup_dict:
            word = lookup_dict[word.lower()]
        new_words.append(word) 
    return new_words
i=0
for row in inptxt.iterrows():
    #row=removeregex(str(row),regex_pattern)
    #print(row)
    words1=str(row).split()
    lemt2=""
    i+=1
    for word in words1:
        word = word.lower()
        word2 = remreg(word)
        #if i==97:
            #print(word2)
        word3 = lem.lemmatize(str(word2),"v")
        lemt1= "".join(word3)
        lemt2+=lemt1 + " "
    lemt3=lemt3+lemt2+'\n'
    row=removenoise(str(lemt2))
    row=lookupwords(row)
    inptxt1=inptxt1+str(row)+'\n'

  



In [161]:
print(inptxt1)

['1', 'testdescription', 'verify', 'select', 'assignment', 'name', 'field', 'max', 'length', '100', 'charsverify', 'name', 'create', 'assignmentset', 'policiesreview', 'assign', 'page', 'name', '1', 'dtype', 'object']
['2', 'testdescription', 'verify', 'direct message', 'student', 'instructions', 'field', 'max', 'length', '4000', 'charsverify', 'instruction', 'create', 'assignmentset', 'policiesreview', 'assign', 'page', 'name', '2', 'dtype', 'object']
['3', 'testdescription', 'verify', 'zero', 'state', 'create', 'assignment', 'page', 'name', '3', 'dtype', 'object']
['4', 'testdescription', 'verify', 'select', 'question', 'source', 'modal', 'select', 'textbook', 'assignment', 'create', 'create', 'question', 'options', 'display', 'name', '4', 'dtype', 'object']
['5', 'testdescription', 'select', 'textbook', 'verify', 'flow', 'display', 'table', 'content', 'chaptersquestion', 'bank', 'select', 'question', 'select', 'chapters', 'list', 'name', '5', 'dtype', 'object']
['6', 'testdescriptio

In [112]:
import nltk
nltk.download('averaged_perceptron_tagger')
  


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MA40006249\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [100]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MA40006249\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [61]:
print(tokens[1])

['(2,', 'TestDescription', 'Verify', 'direct message', 'student', 'instructions', 'field.', 'Max', '...', 'Name:', '2,', 'dtype:', 'object)']


In [51]:
print(words)

[]


In [117]:
print(lemt3)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [103]:
print(tfidfval)

  (0, 17)	1.0
  (1, 0)	1.0
  (2, 3)	1.0
  (3, 11)	1.0
  (4, 14)	1.0
  (5, 9)	1.0
  (6, 17)	1.0
  (7, 7)	1.0
  (8, 6)	1.0
  (9, 15)	1.0
  (10, 10)	1.0
  (11, 19)	1.0
  (12, 5)	1.0
  (13, 14)	1.0
  (14, 4)	1.0
  (15, 12)	1.0
  (16, 18)	1.0
  (17, 5)	1.0
  (18, 2)	1.0
  (19, 8)	1.0
  (20, 16)	1.0
  (21, 1)	1.0
  (22, 13)	1.0


In [106]:
len(word_vectors)

24

In [128]:
print(inptxt)

                                                                                                                                                                                                                                                                                                                                            TestDescription
DS_TEST_ID                                                                                                                                                                                                                                                                                                                                                 
1           verifying 'selecting  the assignment name field. Max length 100 chars.Verify the name from create assignment/set policies/review & assign pages.                                                                                                                                                    

In [19]:
inptxt1.shape

(206, 2)

In [31]:
a = inptxt1.columns[[0]]
type(a)

pandas.core.indexes.base.Index

In [125]:
print(inptxt)

                                                                                                                                                                                                                                                                                                                                            TestDescription
DS_TEST_ID                                                                                                                                                                                                                                                                                                                                                 
1           verifying 'selecting  the assignment name field. Max length 100 chars.Verify the name from create assignment/set policies/review & assign pages.                                                                                                                                                    

In [36]:
b = inptxt1.iterrows()
print(b)

<generator object DataFrame.iterrows at 0x000002DD8CE236D0>


In [127]:
words = ['(1,', 'TestDescription', 'verifying', 'assignment', 'name', 'field.', 'Max', 'leng...', 'Name:', '1,', 'dtype:', 'object)']

for word in words:
    worddrd = lem.lemmatize(word,"v")
    print(worddrd)

(1,
TestDescription
verify
assignment
name
field.
Max
leng...
Name:
1,
dtype:
object)


In [103]:
print(remreg("'kjbnihb"))

kjbnihb
